In [1]:
import cv2
import time

In [2]:
thres=0.5
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

True

In [3]:
classNames = []
classFile = 'coco.names'
with open(classFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(classNames)

['person', 'bottle', 'book', 'cell phone', 'eye glasses', 'laptop', 'orange', 'chair', 'clock', 'cup', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'plate', 'wine glass', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'mouse', 'remote', 'keyboard', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [4]:
configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'

In [5]:
net = cv2.dnn_DetectionModel(weightsPath, configPath)
net.setInputSize(320, 320)
net.setInputScale(1.0 / 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)

< cv2.dnn.Model 0000025F7EDA42D0>

In [6]:
start_time = time.time()
time_limit = 15

In [7]:
speed_start_time = time.time()
frame_count = 0
total_detection_time = 0

In [8]:
fps_start_time = time.time()
fps_frame_count = 0

In [9]:
while True:
    elapsed_time = time.time() - start_time
    if elapsed_time > time_limit:
        print("Time limit reached. Exiting.")
        break
        
    success, img = cap.read()
    if not success:
        print("Error: Unable to read a frame from the video capture.")
        break

    # Measure detection time
    detection_start_time = time.time()
    
    # Detect objects
    try:
        classIds, confs, bbox = net.detect(img, confThreshold=thres)
        detection_time = time.time() - detection_start_time
        total_detection_time += detection_time
        
    except cv2.error as e:
        print(f"Error during detection: {e}")
        break

    if len(classIds) != 0:
        for classId, confidence, box in zip(classIds.flatten(), confs.flatten(), bbox):
            cv2.rectangle(img, box, color=(0, 255, 0), thickness=2)
            cv2.putText(img, classNames[classId - 1].upper(), (box[0] + 10, box[1] + 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(img, str(round(confidence * 100, 2)), (box[0] + 200, box[1] + 30), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            


    # Show the output
    cv2.imshow("Output", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    frame_count += 1

Time limit reached. Exiting.


In [10]:
# Release the video capture object and close windows
cap.release()
# time.sleep(15)
cv2.destroyAllWindows()

In [11]:
# Calculate performance metrics
end_time = time.time()
total_time = end_time - start_time
fps = frame_count / total_time
average_detection_time = total_detection_time / frame_count

In [12]:
print(f"Total Frames: {frame_count}")
print(f"Total Time: {total_time:.2f} seconds")
print(f"FPS: {fps:.2f}")
print(f"Average Detection Time: {average_detection_time:.4f} seconds")

Total Frames: 126
Total Time: 15.47 seconds
FPS: 8.15
Average Detection Time: 0.0996 seconds
